In [1]:
# Cell 0: Import libraries and setup
from breeze_connect import BreezeConnect
import pandas as pd
import time
import os
from datetime import datetime, date, timedelta
import functions as f
import ast
from selenium import webdriver
from pyotp import TOTP
import urllib
import json

In [2]:
# Cell 1: Auto-login to ICICI Breeze API
line = ast.literal_eval(f.get_line('D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/login_credential.txt', 8).strip())
date = line['date']

today = datetime.now().date()
today_date = f'{today.day}-{today.month}-{today.year}'

if today_date != date:

    def autologin():

        api_key = str(ast.literal_eval(f.get_line('D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/login_credential.txt', 1).strip())['api_key'])
        api_secret = str(ast.literal_eval(f.get_line('D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/login_credential.txt', 2).strip())['api_secret'])
        password = str(ast.literal_eval(f.get_line('D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/login_credential.txt', 4).strip())['password'])
        userID = str(ast.literal_eval(f.get_line('D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/login_credential.txt', 5).strip())['userID'])
        totp = str(ast.literal_eval(f.get_line('D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/login_credential.txt', 6).strip())['totp'])

        # ----------------- Selenium -----------------

        browser = webdriver.Chrome()
        browser.get("https://api.icicidirect.com/apiuser/login?api_key="+urllib.parse.quote_plus(api_key))
        browser.implicitly_wait(5)
        breeze = BreezeConnect(api_key=api_key)
        username = browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[1]/div[2]/div/div[1]/input')
        password_field = browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[1]/div[2]/div/div[3]/div/input')

        username.send_keys(userID)
        password_field.send_keys(password)

        # Checkbox
        browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[1]/div[2]/div/div[4]/div/input').click()
        # Click Login Button
        browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[1]/div[2]/div/div[5]/input[1]').click()
        time.sleep(2)
        pin = browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[2]/div/div[2]/div[2]/div[3]/div/div[1]/input')

        totp = TOTP(totp)
        token = totp.now()
        pin.send_keys(token)
        browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[2]/div/div[2]/div[2]/div[4]/input[1]').click()
        time.sleep(1)

        # Check if 'apisession=' exists in the URL
        if 'apisession=' in browser.current_url:
            temp_token = browser.current_url.split('apisession=')[1][:8]
            # Save in Database or text File
            print('temp_token', temp_token)
            f.update_line('D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/login_credential.txt', 7, {'session_key': temp_token})

            breeze.generate_session(api_secret=api_secret, session_token=temp_token)
            print(breeze.get_funds())
        else:
            print("Error: 'apisession=' not found in the URL.")

        browser.quit()

    autologin()

    f.update_line('D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/login_credential.txt', 8, {'date': today_date})

In [3]:
# Cell 2: Initialize Breeze connection with saved credentials
api_key = str(ast.literal_eval(f.get_line('D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/login_credential.txt', 1).strip())['api_key'])
api_secret = str(ast.literal_eval(f.get_line('D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/login_credential.txt', 2).strip())['api_secret'])
session_key = str(ast.literal_eval(f.get_line('D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/login_credential.txt', 7).strip())['session_key'])

breeze = BreezeConnect(api_key=api_key)
breeze.generate_session(api_secret=api_secret, session_token=session_key)

# print(breeze.get_funds())

In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [6]:
df = pd.read_csv("D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/master_list_files/NSEScripMaster.csv")
df = df[df['Series'] == 'EQ']


In [7]:
nifty_comprehensive_df = pd.read_csv('D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/nifty_50_componants_data_from_tradingview.csv', index_col=False)


In [8]:
# # ----------------------------------- find stock in masterlist file -----------------------------------#

n = 0

# Nifty 50 symbols to find
nifty_50 = nifty_comprehensive_df['Symbol'].values.tolist()
nifty_50 = [symbol.replace("_", "-") for symbol in nifty_50]

x = 0
# Fix: Iterate over the actual values in the list, not indices
for stock in nifty_50:
    if stock in df["ExchangeCode"].values:
        x += 1
        a = print(x, stock, "\tFound") if len(stock) > 4 else print(x, stock, "\t\tFound")
        n += 1
    else:
        x += 1
        print(x, stock, "\tNot found")

print(f"Total {n} Nifty 50 stocks found in the DataFrame.")

1 RELIANCE 	Found
2 HDFCBANK 	Found
3 TCS 		Found
4 BHARTIARTL 	Found
5 ICICIBANK 	Found
6 SBIN 		Found
7 INFY 		Found
8 BAJFINANCE 	Found
9 HINDUNILVR 	Found
10 ITC 		Found
11 LT 		Found
12 MARUTI 	Found
13 M&M 		Found
14 HCLTECH 	Found
15 KOTAKBANK 	Found
16 SUNPHARMA 	Found
17 ULTRACEMCO 	Found
18 AXISBANK 	Found
19 BAJAJFINSV 	Found
20 NTPC 		Found
21 TITAN 	Found
22 ETERNAL 	Found
23 ADANIPORTS 	Found
24 ONGC 		Found
25 BEL 		Found
26 ADANIENT 	Found
27 JSWSTEEL 	Found
28 POWERGRID 	Found
29 WIPRO 	Found
30 TATAMOTORS 	Found
31 BAJAJ-AUTO 	Found
32 ASIANPAINT 	Found
33 COALINDIA 	Found
34 NESTLEIND 	Found
35 TATASTEEL 	Found
36 JIOFIN 	Found
37 GRASIM 	Found
38 EICHERMOT 	Found
39 SBILIFE 	Found
40 TRENT 	Found
41 HINDALCO 	Found
42 HDFCLIFE 	Found
43 TECHM 	Found
44 CIPLA 	Found
45 SHRIRAMFIN 	Found
46 APOLLOHOSP 	Found
47 DRREDDY 	Found
48 TATACONSUM 	Found
49 HEROMOTOCO 	Found
50 INDUSINDBK 	Found
Total 50 Nifty 50 stocks found in the DataFrame.


In [9]:
ax1 = df[df['ExchangeCode'].isin(nifty_50)].reset_index(drop=True)
ax1

,Token,ShortName,Series,CompanyName,ticksize,Lotsize,DateOfListing,DateOfDeListing,IssuePrice,FaceValue,ISINCode,52WeeksHigh,52WeeksLow,LifeTimeHigh,LifeTimeLow,HighDate,LowDate,Symbol,InstrumentType,PermittedToTrade,IssueCapital,WarningPercent,FreezePercent,CreditRating,IssueRate,IssueStartDate,InterestPaymentDate,IssueMaturityDate,BoardLotQty,Name,ListingDate,ExpulsionDate,ReAdmissionDate,RecordDate,ExpiryDate,NoDeliveryStartDate,NoDeliveryEndDate,MFill,AON,ParticipantInMarketIndex,BookClsStartDate,BookClsEndDate,EGM,AGM,Interest,Bonus,Rights,Dividends,LocalUpdateDateTime,DeleteFlag,Remarks,NormalMarketStatus,OddLotMarketStatus,SpotMarketStatus,AuctionMarketStatus,NormalMarketEligibility,OddLotlMarketEligibility,SpotMarketEligibility,AuctionlMarketEligibility,MarginPercentage,ExchangeCode
0,10604,BHAAIR,EQ,BHARTI AIRTEL LTD,0.01,1,NaN,NaN,0,5.0,INE397D01024,2045.80,1511.00,2045.80,9.31,NaN,NaN,NaN,0,0,0,2104.30,1721.70,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,BHARTIARTL
1,10999,MARUTI,EQ,MARUTI SUZUKI INDIA LTD,0.01,1,NaN,NaN,0,5.0,INE585B01010,15384.00,10725.00,15384.00,155.00,NaN,NaN,NaN,0,0,0,16604.00,13586.00,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,MARUTI
2,11483,LARTOU,EQ,LARSEN AND TOUBRO LIMITED,0.01,1,NaN,NaN,0,2.0,INE018A01030,3963.50,2965.30,3963.50,15.57,NaN,NaN,NaN,0,0,0,3893.60,3185.80,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,LT
3,11532,ULTCEM,EQ,ULTRATECH CEMENT LIMITED,0.01,1,NaN,NaN,0,10.0,INE481G01011,13097.00,10047.85,13097.00,245.25,NaN,NaN,NaN,0,0,0,13631.00,11153.00,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,ULTRACEMCO
4,11536,TCS,EQ,TATA CONSULTANCY SERVICES LTD,0.01,1,NaN,NaN,0,1.0,INE467B01029,4549.35,2991.60,4592.25,103.84,NaN,NaN,NaN,0,0,0,3436.60,2811.80,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,TCS
5,11630,NTPC,EQ,NTPC LIMITED,0.01,1,NaN,NaN,0,10.0,INE733E01010,448.45,292.80,448.45,58.33,NaN,NaN,NaN,0,0,0,364.15,297.95,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NTPC
6,11723,JSWSTE,EQ,JSW STEEL LIMITED,0.01,1,NaN,NaN,0,1.0,INE019A01038,1115.50,880.00,1115.50,0.19,NaN,NaN,NaN,0,0,0,1208.90,989.10,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,JSWSTEEL
7,1232,GRASIM,EQ,GRASIM INDUSTRIES LIMITED,0.01,1,NaN,NaN,0,2.0,INE047A01021,2898.40,2276.95,2898.40,20.38,NaN,NaN,NaN,0,0,0,3078.90,2519.10,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,GRASIM
8,1333,HDFBAN,EQ,HDFC BANK LIMITED,0.01,1,NaN,NaN,0,1.0,INE040A01034,1018.85,806.50,1018.85,1.20,NaN,NaN,NaN,0,0,0,1064.55,871.05,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,HDFCBANK
9,1348,HERHON,EQ,HERO MOTOCORP LIMITED,0.01,1,NaN,NaN,0,2.0,INE158A01026,6246.25,3344.00,6246.25,17.00,NaN,NaN,NaN,0,0,0,5832.00,4772.00,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,HEROMOTOCO


In [10]:
# save ax1 to csv in the D:\Programming\Download_Backtest_Deploy_data\1__Download\1__Download_data_ICICI_via_API location
ax1.to_csv('D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/nifty_50_componants_data_masterlist.csv', index=False)